In [ ]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install refractio
!pip install refractml

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")df = snowflake.get_dataframe("HR_ANALYTICS_V")
df

In [2]:
from refractio import snowflake
snowflake.get_connection(connection_name="HR_Attrition_Data")

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Exception occurred in getting snowflake connection: 'connectionSources'


In [3]:
df = snowflake.get_dataframe("HR_Attrition_Data")
df

,EMPLOYEE_ID,SALARY,JOB_SATISFACTION,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,OVER_TIME,RELATIONSHIP_SATISFACTION,CHURN,MAPPED_ROLE_CLEAN,HOSPITAL_TYPE,...,JOB_STARTDATE,JOB_ENDDATE,BIRTH_YEAR,AGE,SENIORITY,OVERTIME_HOURS,WORK_LIFE_BALANCE,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,USER_ID
0,94215814,53239.07,Low,11,Excellent,True,Medium,1,nurse,Acute Care Hospitals,...,2016-11-01,2017-06-01,1965,59,4,13,Best,Travel_Rarely,High,147501806
1,20227353,29277.97,Medium,11,Outstanding,False,Low,1,nurse,Acute Care Hospitals,...,2015-07-01,2016-06-01,1993,31,1,15,Good,Travel_Rarely,High,46143395
2,72815065,43763.54,Medium,18,Excellent,True,Medium,1,technologist,Acute Care Hospitals,...,2010-07-01,2012-05-01,1985,39,2,0,Better,Travel_Rarely,High,567514867
3,15048356,24728.57,Medium,12,Low,False,Very High,0,nurse,Acute Care Hospitals,...,2014-05-01,2015-09-01,1992,32,1,9,Good,Travel_Rarely,Very High,65551252
4,91151756,46819.01,Low,19,Good,False,Low,0,technologist,Acute Care Hospitals,...,2009-06-01,None,1983,41,2,13,Bad,Travel_Rarely,Very High,84119911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,11954788,30934.15,Medium,14,Excellent,True,Medium,1,nurse,Psychiatric,...,2019-09-01,2020-02-01,1996,28,1,2,Best,Travel_Rarely,Low,163926894
299996,65800600,69288.47,Medium,18,Good,True,Very High,1,nurse,Acute Care Hospitals,...,2019-04-01,2020-02-01,1996,28,2,3,Best,Travel_Rarely,Low,313934824
299997,78774518,53051.71,High,20,Outstanding,False,Low,0,emt,Acute Care Hospitals,...,2018-12-01,None,1991,33,1,8,Best,Travel_Rarely,High,812455262
299998,53352261,72186.39,Low,23,Low,False,Very High,0,technologist,Acute Care Hospitals,...,2020-09-01,None,1995,29,2,9,Best,Travel_Rarely,Medium,713192325


In [4]:
print(df.isnull().sum())

EMPLOYEE_ID                      0
SALARY                           0
JOB_SATISFACTION                 0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
OVER_TIME                        0
RELATIONSHIP_SATISFACTION        0
CHURN                            0
MAPPED_ROLE_CLEAN                0
HOSPITAL_TYPE                    0
HOSPITAL_OWNERSHIP               0
COMPANY_NAME                     0
CITY                             0
STATE                            0
DISTANCE                         0
DEGREE_CLEAN                     0
SEX                              0
ETHNICITY                        0
MARITAL_STATUS                   0
TENURE_MONTHS                    0
MONTHS_AFTER_COLLEGE             0
SCHOOL_ENDDATE                   0
JOB_STARTDATE                    0
JOB_ENDDATE                  90076
BIRTH_YEAR                       0
AGE                              0
SENIORITY                        0
OVERTIME_HOURS                   0
WORK_LIFE_BALANCE   

In [5]:
df = df.dropna()

In [6]:
print(df.isnull().sum())

EMPLOYEE_ID                  0
SALARY                       0
JOB_SATISFACTION             0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
OVER_TIME                    0
RELATIONSHIP_SATISFACTION    0
CHURN                        0
MAPPED_ROLE_CLEAN            0
HOSPITAL_TYPE                0
HOSPITAL_OWNERSHIP           0
COMPANY_NAME                 0
CITY                         0
STATE                        0
DISTANCE                     0
DEGREE_CLEAN                 0
SEX                          0
ETHNICITY                    0
MARITAL_STATUS               0
TENURE_MONTHS                0
MONTHS_AFTER_COLLEGE         0
SCHOOL_ENDDATE               0
JOB_STARTDATE                0
JOB_ENDDATE                  0
BIRTH_YEAR                   0
AGE                          0
SENIORITY                    0
OVERTIME_HOURS               0
WORK_LIFE_BALANCE            0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
USER_ID                      0
dtype: i

In [7]:
df = df.drop(["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "SCHOOL_ENDDATE"], axis = 1)

In [9]:
CATEGORICAL_COLUMNS = ["MAPPED_ROLE_CLEAN","SEX", "ETHNICITY","HOSPITAL_TYPE", "HOSPITAL_OWNERSHIP","COMPANY_NAME","CITY","STATE","DISTANCE", 
                       "DEGREE_CLEAN","BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARITAL_STATUS","OVER_TIME","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE","OVERTIME_HOURS","PERCENTAGE_SALARY_HIKE"]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [10]:
# Filter feature columns
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS
feature_columns = [col for col in feature_columns if col in df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in df.columns]
 
# Split data into features and labels
X = df[feature_columns]
y = df[LABEL_COLUMNS].values.ravel()  # Flatten to 1D array for consistency

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
 
# Define transformers
categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)
 
numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)
 
# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)
 
# Create pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

pipeline.fit(X_train, y_train)
result = pipeline.predict(X_test)

In [14]:
result

array([1, 0, 0, ..., 1, 0, 0])

In [15]:
from joblib import dump, load
filename = "HR_Attrition.joblib"
dump(pipeline, filename)

['HR_Attrition.joblib']

In [16]:
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)

In [17]:
from refractml import *

from refractml.constants import MLModelFlavours

In [18]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data_json = eval(payload)
    data = pd.DataFrame([data_json])
    prediction = str(model.predict(data)[0])
    return prediction

In [19]:
import requests
payload = str(X_test.iloc[123].to_dict())
req = requests.Request()
req.json = {"payload": payload}

print(score(pipeline, req))

1


In [20]:
req.json

{'payload': "{'MAPPED_ROLE_CLEAN': 'nurse', 'SEX': 'M', 'ETHNICITY': 'Black', 'HOSPITAL_TYPE': 'Acute Care Hospitals', 'HOSPITAL_OWNERSHIP': 'Government - Hospital District or Authority', 'COMPANY_NAME': 'SSM Health Care Corp.', 'CITY': 'Madison', 'STATE': 'WI', 'DISTANCE': '>10 miles', 'DEGREE_CLEAN': 'Professional Certificate', 'BUSINESS_TRAVEL': 'Travel_Frequently', 'ENVIRONMENT_SATISFACTION': 'High', 'JOB_SATISFACTION': 'Very High', 'MARITAL_STATUS': 'Divorced', 'OVER_TIME': False, 'PERFORMANCE_RATING': 'Outstanding', 'RELATIONSHIP_SATISFACTION': 'High', 'WORK_LIFE_BALANCE': 'Better', 'SALARY': 57183.9, 'SENIORITY': 1, 'TENURE_MONTHS': 1, 'MONTHS_AFTER_COLLEGE': 38, 'BIRTH_YEAR': 1990, 'AGE': 34, 'OVERTIME_HOURS': 20, 'PERCENTAGE_SALARY_HIKE': 11}"}

In [21]:
## registering the model in refract.
tmp = register_model(pipeline, 
               score, 
               name="HR_ATTRITION_ML_MODEL", 
               description="Analyzing_HR_Attrition_trained_using _ml",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install seaborn \\n pip install snowflake-connector-python[pandas] \\n pip install joblib==1.3.2 scikit-learn=1.3.2",
               y_true=y_test,
               y_pred=y_pred, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True,
               prob=y_prob,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['NOT LEFT','LEFT'],
               kyd=True, kyd_score = True)


Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


In [22]:
payload  = {"payload": X_test.iloc[0].to_dict()}
payload

{'payload': {'MAPPED_ROLE_CLEAN': 'nurse',
  'SEX': 'F',
  'ETHNICITY': 'API',
  'HOSPITAL_TYPE': 'Acute Care Hospitals',
  'HOSPITAL_OWNERSHIP': 'Government - Local',
  'COMPANY_NAME': 'Adventist Health System Sunbelt Healthcare Corp.',
  'CITY': 'Deltona',
  'STATE': 'FL',
  'DISTANCE': '<2 miles',
  'DEGREE_CLEAN': 'Associates Degree',
  'BUSINESS_TRAVEL': 'Travel_Rarely',
  'ENVIRONMENT_SATISFACTION': 'Very High',
  'JOB_SATISFACTION': 'Low',
  'MARITAL_STATUS': 'Divorced',
  'OVER_TIME': True,
  'PERFORMANCE_RATING': 'Good',
  'RELATIONSHIP_SATISFACTION': 'High',
  'WORK_LIFE_BALANCE': 'Bad',
  'SALARY': 83361.28,
  'SENIORITY': 3,
  'TENURE_MONTHS': 1,
  'MONTHS_AFTER_COLLEGE': 82,
  'BIRTH_YEAR': 1992,
  'AGE': 32,
  'OVERTIME_HOURS': 17,
  'PERCENTAGE_SALARY_HIKE': 24}}

In [ ]:
headers={"Content-type":"application/json"}
url = "http://svc-8164792a-076f-4926-b10f-f0d857945438:5001/humanresourceattritionmlmodel/f0acb182-64a2-4f2c-bd03-221a994b0641/score"
data={"payload": payload}
response = requests.post(url, json=data, headers=headers) 
response.json()